In [ ]:
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
import kagglehub
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader, Subset
import torchvision.models as models
from tqdm import tqdm
import timm
import random
from torch.amp import autocast, GradScaler



# Download dataset
path = kagglehub.dataset_download("steubk/wikiart")

print("Path to dataset files:", path)

# GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# Dataset path
dataset_path = "/kaggle/input/wikiart" #


# Image transformation
transform = transforms.Compose([
    transforms.Resize(518),
    #transforms.Resize(256), # for Resnet
    transforms.CenterCrop(518),
    #transforms.CenterCrop(224), # for Resnet
    #transforms.RandomHorizontalFlip(), # overfitting (not used in final experiement)
    #transforms.RandomRotation(15), # overfitting (not used in final experiement)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Hyperparameters
batch_size = 64
epochs = 3
learning_rate = 0.001


# Load dataset
full_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Subset_size = number of images to train and evaluate. Used for short time partial training of the dataset
subset_size = 80020
small_dataset = Subset(full_dataset, torch.randperm(len(full_dataset))[:subset_size])

# Split into 80% training, 20% testing
train_size = int(0.8 * subset_size) # 80% of images for training
test_size = subset_size - train_size # 20% of images for testing
train_dataset, test_dataset = random_split(small_dataset, [train_size, test_size]) # randomly distribute images for training and testing

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Load DINOv2 ViT-Large
#backbone =models.resnet18(weights=models.ResNet18_Weights.DEFAULT) # Load Resnet for comparison
backbone = timm.create_model('vit_large_patch14_dinov2.lvd142m', pretrained=True)

# backbone (time saving)
for param in backbone.parameters():
    #param.requires_grad = True # unfrozen backbone
     param.requires_grad = False # fronzen backbone

# classification head
model = nn.Sequential(
    backbone,
    nn.Dropout(0.5), #dropout added for overfitting issue
    nn.Linear(1024, len(full_dataset.classes))
    #nn.Linear(1000, len(full_dataset.classes)) # for ResNet
).to(device)

# mixed precision
scaler = GradScaler()


# Training setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# Train and Evaluate
print("Starting training...")
results = []
train_losses = []
test_accuracies = []


    # training
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    loop = tqdm(train_loader, desc=f" Epoch {epoch+1}/{epochs}", leave= False)
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        with autocast(device_type= 'cuda', dtype= torch.float16):  # enable mixed precision
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()  # gradient scaling process
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        loop.set_postfix(loss=loss.item()) # view running loss in the progress bar

    avg_loss = running_loss / len(train_loader)
    train_losses.append(avg_loss)


    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct * 100 / total
    test_accuracies.append(accuracy)
    print(f" Epoch {epoch+1}: Loss={avg_loss:.4f}, Accuracy={accuracy:.2f}%")

results.append((train_losses, test_accuracies)) # could be used for result interpretation